# Preguntas y respuestas (Consultas en vivo)

# ENTREGABLE 2

## Trigger
1. Crea un trigger que registre en una tabla de monitoreo cada vez que un producto supere las 200.000 unidades vendidas acumuladas.

El trigger debe activarse después de insertar una nueva venta y registrar en la tabla el ID del producto, su nombre, la nueva cantidad total de unidades vendidas, y la fecha en que se superó el umbral.

**Solución** 

__Se creó tabla de auditoria en la base de datos ya existente:__

```sql
USE sales_company;

DROP TABLE IF EXISTS product_monitoring;
CREATE TABLE product_monitoring (
    ID INT AUTO_INCREMENT PRIMARY KEY,
    ProductID INT,
    ProductName VARCHAR(255),
    TotalSold INT,
    ThresholdDate DATETIME DEFAULT CURRENT_TIMESTAMP
);
```
__Comando sql para crear trigger solicitado__


```sql
DELIMITER $$
CREATE TRIGGER trigger_product_threshold_200k --nombre del trigger
AFTER INSERT ON sales -- cuado se ejecuta? luego de cada insert en la tabla sales
FOR EACH ROW
BEGIN
    DECLARE total_sales INT; 
    -- setea en la variable declarada a partir de la sumatoria de la columna quantity de la tabla sales para el producto que se acaba de insertar
    SELECT SUM(Quantity)
    INTO total_sales
    FROM sales
    WHERE ProductID = NEW.ProductID;

    IF total_sales > 200000 THEN
    -- si el total de ventas es mayor a doscientos mil, se verifica si ya existe un registro de sobrepaso para el producto recién ingresado y si no existe , se inserta registro de auditoria
        IF NOT EXISTS (
            SELECT 1 FROM product_monitoring
            WHERE ProductID = NEW.ProductID
        ) THEN
            INSERT INTO product_monitoring (
                ProductID,
                productName,
                TotalSold
            )
            -- el select recupera la info que se insertará en la tabla de auditoria (product_monitoring) a partir de la tabla prodctos y del total de venas guardado como variable
            SELECT 
                ProductID,
                ProductName,
                total_sales
            FROM products
            WHERE p.ProductID = NEW.ProductID;
        END IF;
    END IF;

END$$

DELIMITER ;
```

**NOTA** El trigger se ejecuta con cada inserción en la tabla sales. Por eso se decidió ejecutar su creación a posteriori de la carga inicial de datos, para evitar ejecuciones innecesarias durante el poblamiento inicial de la base.

##  Registro
Registra una venta correspondiente al vendedor con ID 9, al cliente con ID 84, del producto con ID 103, por una cantidad de 1.876 unidades y un valor de 1200 unidades.

Consulta la tabla de monitoreo, toma captura de los resultados y realiza un análisis breve de lo ocurrido.

In [1]:
import utils.sql_utils as sql_utils
import utils.notebook_utils as notebook_utils

notebook_utils.print_colored('INSERTING NEW SALE RECORD', 'orange')
query = """
INSERT INTO sales (salesID, salesPersonID, customerID, productID, quantity, totalPrice, salesDate)
SELECT 
    COALESCE(MAX(salesID), 0) + 1,
    9,
    84,
    103,
    1876,
    1200,
    '2025-06-10 10:00:00'
FROM sales;
"""

sql_utils.run_non_select_query(query=query)

notebook_utils.print_colored('Check new register on monitoring db', 'green')

query = """
SELECT * FROM product_monitoring
WHERE productID = 103;
"""

notebook_utils.print_colored('Monitoring table data', 'orange')
sql_utils.run_query(query=query)


closing connection, no errors in the query


,ID,ProductID,ProductName,TotalSold,ThresholdDate
0,1,103,Cream Of Tartar,200002,2025-06-11 00:58:05


**Análisis** :
El registro insertado provocó que el acumulado de unidades vendidas para el producto con ID 103 superara el umbral de 200.000 unidades. Dado que esta es la condición definida para la activación del trigger __trigger_product_threshold_200k__, el mismo trigger se ejecutó automáticamente. Como resultado, se insertó un registro en la tabla de auditoría __product_monitoring__ con los datos correspondientes al producto. Este comportamiento ocurrió de forma transparente para el usuario, evidenciando el funcionamiento correcto del trigger ante el cumplimiento de la condición.

## Optimización
1. Selecciona dos consultas del avance 1 y crea los índices que consideres más adecuados para optimizar su ejecución.

2. Prueba con índices individuales y compuestos, según la lógica de cada consulta. Luego, vuelve a ejecutar ambas consultas y compara los tiempos de ejecución antes y después de aplicar los índices. Finalmente, describe brevemente el impacto que tuvieron los índices en el rendimiento y en qué tipo de columnas resultan más efectivos para este tipo de operaciones.

**Query 1** (~12 segundos en entorno kernel).
-- La siguiente consulta calcula:

* los 5 productos más vendidos (top_products).

* Para cada uno, se agrupan las ventas por vendedor y se calcula volumen individual de ventas (seller_sales).

* Ranking de cada vendedor por producto en función del total vendido, para luego obtener el top 1 de cada producto (ranked_sellers).

In [12]:
import utils.sql_utils as sql_utils

query= """
WITH top_products AS (
    SELECT 
        p.productID AS product_id,
        p.productName AS product_name,
        SUM(s.quantity) AS total_quantity
    FROM products p
    JOIN sales s ON p.productID = s.productID
    GROUP BY p.productID, p.productName
    ORDER BY total_quantity DESC
    LIMIT 5
),
seller_sales AS (
    SELECT 
        tp.product_id,
        tp.product_name,
        tp.total_quantity AS total_sold,
        s.salesPersonID AS seller_id,
        SUM(s.quantity) AS seller_quantity
    FROM top_products tp
    JOIN sales s ON tp.product_id = s.productID
    GROUP BY tp.product_id, tp.product_name, s.salesPersonID
),
ranked_sellers AS (
    SELECT 
        *,
        RANK() OVER (PARTITION BY product_id ORDER BY seller_quantity DESC) AS seller_rank
    FROM seller_sales
)
SELECT 
    rs.product_id,
    rs.product_name,
    rs.seller_id,
    rs.total_sold,
    CONCAT( e.FirstName ,' ', e.LastName) AS seller_name,
    rs.seller_quantity
FROM ranked_sellers rs
JOIN employees e ON rs.seller_id = e.employeeID
WHERE seller_rank = 1
ORDER BY seller_quantity DESC;
"""

sql_utils.run_query(query=query)
# Descomentar para ejecutar EXPLAIN
#explain_query = """ EXPLAIN """ + query
#sql_utils.run_query(query=explain_query)

,product_id,product_name,seller_id,total_sold,seller_name,seller_quantity
0,47,Thyme - Lemon; Fresh,21,198567.0,Devon Brewer,11050.0
1,161,Longos - Chicken Wings,10,199659.0,Jean Vang,10785.0
2,280,Onion Powder,21,198163.0,Devon Brewer,10570.0
3,103,Cream Of Tartar,9,200002.0,Daphne King,10551.0
4,179,Yoghurt Tubes,9,199724.0,Daphne King,10285.0


* Índices aplicados

Se crearon los siguientes índices para optimizar las cláusulas JOIN, GROUP BY y ORDER BY que son las más costosas:

In [ ]:
-- products: la clave usada en JOIN y GROUP BY 
CREATE INDEX idx_products_productID_name ON products(productID, productName);

-- sales: combinando JOIN + WHERE + agregación
CREATE INDEX idx_sales_productID_salesPersonID_quantity ON sales(productID, salesPersonID, quantity);

Query para creación de índices 

In [3]:
import utils.sql_utils as sql_utils


query= """
CREATE INDEX idx_products_productID_name ON products(productID, productName);

-- sales: combinando JOIN + WHERE + agregación
CREATE INDEX idx_sales_productID_salesPersonID_quantity ON sales(productID, salesPersonID, quantity);
"""

sql_utils.run_non_select_query(query=query)

closing connection, no errors in the query


MySQLInterfaceError: Commands out of sync; you can't run this command now

### Impacto observado

Tras la aplicación de los índices:

* El tiempo de ejecución se redujo en casi un 75%.

* Se evitan full scan en las tablas sales y products, mediante el uso de índices.

* Agregación y ordenamiento se benefician del uso de índices compuestos que combinan las columnas más utilizadas. 

**Optimización 2** (~15 segundos en entorno kernel)

La siguinte consulta calcula:
* los 5 productos más vendidos (top_products). 
* Para esos productos, se cuentan la cantidad de clientes únicos que compraron dichos productos (customer_counts).
* Sumatoria de clientes (total_customers).
* Porcentaje de clientes únicos que compraron cada producto top respecto al total.

In [11]:
import utils.sql_utils as sql_utils

query = """
WITH top_products AS (
    SELECT 
        p.productID AS product_id,  
        p.productName AS product_name,
        SUM(s.quantity) AS total_quantity
    FROM products p
    JOIN sales s ON p.productID = s.productID
    GROUP BY p.productID, p.productName
    ORDER BY total_quantity DESC
    LIMIT 5
),
customer_counts AS (
    SELECT
        tp.product_id,
        COUNT(DISTINCT s.customerID) AS unique_customers
    FROM top_products tp
    JOIN sales s ON tp.product_id = s.productID
    GROUP BY tp.product_id
),
total_customers AS (
    SELECT COUNT(DISTINCT customerID) AS total_customers
    FROM sales
)
SELECT
    cc.product_id,
    tp.product_name,
    cc.unique_customers,
    tc.total_customers,
    ROUND((cc.unique_customers/ tc.total_customers) * 100, 2) AS proportion_percentage
FROM customer_counts cc
JOIN top_products tp ON cc.product_id = tp.product_id
JOIN total_customers tc ON true
ORDER BY cc.unique_customers DESC;
"""
sql_utils.run_query(query=query)

# Descomentar para ejecutar EXPLAIN
#explain_query = """ EXPLAIN """ + query
#sql_utils.run_query(query=explain_query)

,product_id,product_name,unique_customers,total_customers,proportion_percentage
0,161,Longos - Chicken Wings,14252,98759,14.43
1,103,Cream Of Tartar,14247,98759,14.43
2,47,Thyme - Lemon; Fresh,14101,98759,14.28
3,179,Yoghurt Tubes,14066,98759,14.24
4,280,Onion Powder,14058,98759,14.23


* Índices aplicados
Se crearon los siguientes índices para optimizar las cláusulas JOIN, GROUP BY y ORDER BY que generaban cuellos de botella:

In [9]:
import utils.sql_utils as sql_utils
query = """
CREATE INDEX idx_sales_productID_quantity ON sales(productID, quantity);
CREATE INDEX idx_sales_productID_customerID ON sales(productID, customerID);
CREATE INDEX idx_sales_customerID ON sales(customerID);
"""

sql_utils.run_non_select_query(query=query)


Error: 1061 (42000): Duplicate key name 'idx_sales_productID_quantity'
closing connection, no errors in the query


- Si se reejecuta query anterior se observa el cambio en el plan de ejecución y menor tiempo pararesolver consulta

## Impacto observado
Tras la creación de los índices,

- El tiempo de ejecución se redujo casi 5 veces (~3 segundos en entorno kernel).

- Explain muestra que se usarán los índices en casi todos los joins y filtros, esto redunda en su optmización.

- La tabla sales ahora usa range y ref en lugar de ALL (full scan).

- Los índices idx_sales_customerID e idx_sales_productID_customerID están tp.product_id reduce a 5 filas, lo cual permite joins rápidos y precisos (dado que son menos elementos)